In [1]:
import os
os.chdir("C:/Users/david/Desktop/Uni/potato-dry-matter-optics-ml")

from dotenv import load_dotenv

load_dotenv()  # lee el archivo .env

ROBOFLOW_API_KEY = os.environ["ROBOFLOW_API_KEY"]
os.environ["ROBOFLOW_API_KEY"] = ROBOFLOW_API_KEY

# Comprobación rápida
print("ROBOFLOW_API_KEY configurada?:", "ROBOFLOW_API_KEY" in os.environ)


ROBOFLOW_API_KEY configurada?: True


In [2]:
from inference import get_model

# ID del modelo tal como aparece en Roboflow (deploy > Python)
MODEL_ID = "potato-detection-3et6q/11"  # si cambia la versión, actualiza aquí

# Cargar el modelo
model = get_model(MODEL_ID)

print("Modelo cargado correctamente:", MODEL_ID)


ModelDependencyMissing: Your `inference` configuration does not support SAM model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM2 model. Use pip install 'inference[sam]' to install missing requirements.To suppress this warning, set CORE_MODEL_SAM2_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support SAM3 model. Install SAM3 dependencies and set CORE_MODEL_SAM3_ENABLED to True.
ModelDependencyMissing: Your `inference` configuration does not support Gaze Detection model. Use pip install 'inference[gaze]' to install missing requirements.To suppress this warning, set CORE_MODEL_GAZE_ENABLED to False.
ModelDependencyMissing: Your `inference` configuration does not support GroundingDINO model. Use pip install 'inference[grounding-dino]' to install missing requirements.To suppress this warning, set CO

Modelo cargado correctamente: potato-detection-3et6q/11


In [3]:
from PIL import Image
from typing import List, Dict, Any

def run_potato_detection(image_path: str, confidence_threshold: float = 0.4):
    """
    Ejecuta el modelo de Roboflow sobre una imagen y devuelve:
      - raw_result: objeto ObjectDetectionInferenceResponse
      - detections: lista de dicts con clase, confianza y bbox en formato (x1, y1, x2, y2)
    """
    # Cargar imagen (solo para saber tamaño, el modelo puede usar la ruta directamente)
    image = Image.open(image_path).convert("RGB")
    w, h = image.size

    # Inferencia
    raw = model.infer(image_path, confidence=confidence_threshold)

    # Si devuelve lista, nos quedamos con el primer elemento (caso normal)
    raw_result = raw[0] if isinstance(raw, list) else raw

    # >>> AQUÍ ESTABA EL ERROR: raw_result es un objeto, no un dict
    predictions = getattr(raw_result, "predictions", [])

    detections: List[Dict[str, Any]] = []

    for det in predictions:
        # det es un ObjectDetectionPrediction
        cls = det.class_name          # nombre de la clase (tipo de patata)
        conf = det.confidence         # confianza (0-1)
        x = det.x
        y = det.y
        bw = det.width
        bh = det.height

        # Pasar de (centro x,y + ancho/alto) a esquinas (x1, y1, x2, y2)
        x1 = int(x - bw / 2)
        y1 = int(y - bh / 2)
        x2 = int(x + bw / 2)
        y2 = int(y + bh / 2)

        # Clip a los límites de la imagen
        x1 = max(0, x1)
        y1 = max(0, y1)
        x2 = min(w, x2)
        y2 = min(h, y2)

        detections.append({
            "class": cls,
            "confidence": float(conf),
            "bbox_xyxy": (x1, y1, x2, y2),
        })

    return raw_result, detections

In [7]:
IMAGE_PATH = "data/input/raw/test_img1.jpg"  # <-- cambia esto

raw_result, detections = run_potato_detection(IMAGE_PATH, confidence_threshold=0.4)

print("=== OUTPUT BRUTO DEL MODELO ===")
print(raw_result)  # Aquí ves TODO el JSON que devuelve el modelo

print("\n=== DETECCIONES PROCESADAS ===")
for i, det in enumerate(detections, start=1):
    print(f"Patata {i}:")
    print(f"  Clase (tipo): {det['class']}")
    print(f"  Confianza: {det['confidence']:.3f}")
    print(f"  Bounding box (x1, y1, x2, y2): {det['bbox_xyxy']}")


=== OUTPUT BRUTO DEL MODELO ===
visualization=None inference_id=None frame_id=None time=None image=InferenceResponseImage(width=1200, height=1600) predictions=[ObjectDetectionPrediction(x=596.5, y=771.0, width=717.0, height=582.0, confidence=0.7184754610061646, class_name='Sprouted potato', class_confidence=None, class_id=4, tracker_id=None, detection_id='a8fb00b7-519e-4d96-aa23-cc53d898a614', parent_id=None)]

=== DETECCIONES PROCESADAS ===
Patata 1:
  Clase (tipo): Sprouted potato
  Confianza: 0.718
  Bounding box (x1, y1, x2, y2): (238, 480, 955, 1062)


In [8]:
from PIL import Image, ImageDraw, ImageFont
import os

def save_detections_image(image_path: str, detections, output_path: str):
    # Cargar imagen
    image = Image.open(image_path).convert("RGB")
    draw = ImageDraw.Draw(image)

    # Fuente para el texto
    try:
        font = ImageFont.truetype("arial.ttf", 18)
    except:
        font = ImageFont.load_default()

    for det in detections:
        x1, y1, x2, y2 = det["bbox_xyxy"]
        label = f"{det['class']} {det['confidence']:.2f}"

        # Caja
        draw.rectangle([x1, y1, x2, y2], outline="red", width=3)

        # Fondo para texto
        text_x, text_y = x1, max(0, y1 - 22)
        w_text = 8 * len(label)
        h_text = 20
        draw.rectangle(
            [text_x, text_y, text_x + w_text, text_y + h_text],
            fill="red"
        )

        # Texto
        draw.text((text_x + 2, text_y + 2), label, fill="white", font=font)

    # Crear carpeta si no existe
    os.makedirs(os.path.dirname(output_path), exist_ok=True)
    image.save(output_path)
    print(f"Imagen con detecciones guardada en: {output_path}")

In [9]:
OUTPUT_PATH = "data/output/detections/test_img1_detected.jpg"

# Suponiendo que ya has hecho:
# raw_result, detections = run_potato_detection(IMAGE_PATH, confidence_threshold=0.4)

save_detections_image(IMAGE_PATH, detections, OUTPUT_PATH)


Imagen con detecciones guardada en: data/output/detections/test_img1_detected.jpg
